In [12]:

import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time
from pathlib import Path

print("="*70)
print("NOTEBOOK 6: REAL-TIME FATIGUE DETECTION")
print("="*70)
print("\nImports successful! ✓")


NOTEBOOK 6: REAL-TIME FATIGUE DETECTION

Imports successful! ✓


In [13]:
# Load trained model
model_path = Path('../models/improved_cnn_best.keras')

print(f"\nLoading model from: {model_path}")
model = load_model(str(model_path))
print("✓ Model loaded successfully!")

# Load face and eye cascade classifiers (comes with OpenCV)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

print("✓ Face detector loaded!")
print("✓ Eye detector loaded!")
print("\nSystem ready for real-time detection!")



Loading model from: ..\models\improved_cnn_best.keras
✓ Model loaded successfully!
✓ Face detector loaded!
✓ Eye detector loaded!

System ready for real-time detection!


In [14]:
def run_improved_detection(duration=60):
    """
    Enhanced real-time detection with improved eye detection parameters
    """
    CLOSED_THRESHOLD = 5
    
    cap = cv2.VideoCapture(0)
    closed_counter = 0
    frame_count = 0
    start_time = time.time()
    fps = 0
    
    print("\n" + "="*70)
    print("ENHANCED DETECTION STARTED")
    print("="*70)
    print("\nInstructions:")
    print("  - Keep your face straight for best results")
    print("  - Ensure good lighting on your face")
    print("  - Stay 50-100cm from camera")
    print("  - Press 'q' to quit")
    print("  - Press 's' to save screenshot\n")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        elapsed = time.time() - start_time
        if elapsed > duration:
            break
        
        frame_count += 1
        if frame_count % 30 == 0:
            fps = 30 / (time.time() - start_time)
            start_time = time.time()
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # More sensitive face detection
        faces = face_cascade.detectMultiScale(
            gray, 
            scaleFactor=1.1,
            minNeighbors=3,
            minSize=(100, 100)
        )
        
        eyes_closed = False
        eyes_detected = False
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2)
            
            # Expand region for eyes (70% of face instead of 50%)
            roi_gray = gray[y:y+int(h*0.7), x:x+w]
            roi_color = frame[y:y+int(h*0.7), x:x+w]
            
            # More sensitive eye detection parameters
            eyes = eye_cascade.detectMultiScale(
                roi_gray,
                scaleFactor=1.05,
                minNeighbors=3,
                minSize=(20, 20),
                maxSize=(100, 100)
            )
            
            for (ex, ey, ew, eh) in eyes:
                eyes_detected = True
                
                try:
                    eye_img = roi_gray[ey:ey+eh, ex:ex+ew]
                    
                    # Preprocess
                    eye_resized = cv2.resize(eye_img, (128, 128))
                    eye_normalized = eye_resized / 255.0
                    eye_input = np.expand_dims(eye_normalized, axis=(0, -1))
                    
                    # Predict
                    prediction = model.predict(eye_input, verbose=0)[0][0]
                    is_open = prediction > 0.5
                    confidence = prediction if is_open else (1 - prediction)
                    
                    # Draw
                    color = (0, 255, 0) if is_open else (0, 0, 255)
                    label = f"{'OPEN' if is_open else 'CLOSED'} ({confidence*100:.0f}%)"
                    
                    cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), color, 2)
                    cv2.putText(roi_color, label, (ex, ey-5), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
                    
                    if not is_open:
                        eyes_closed = True
                
                except Exception as e:
                    pass
            
            # Warning if no eyes detected
            if not eyes_detected:
                cv2.putText(frame, "WARNING: Eyes not detected", 
                           (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 165, 255), 2)
        
        # Update counter
        if eyes_closed:
            closed_counter += 1
        elif eyes_detected:
            closed_counter = 0
        
        # Alert
        if closed_counter >= CLOSED_THRESHOLD:
            cv2.rectangle(frame, (0, 0), (frame.shape[1], 120), (0, 0, 255), -1)
            cv2.putText(frame, "DROWSINESS ALERT!", (50, 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 3)
            cv2.putText(frame, "WAKE UP!", (50, 100),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2)
        
        # Status display
        if not eyes_detected:
            status = "NO EYES DETECTED"
            status_color = (0, 165, 255)
        elif eyes_closed:
            status = "EYES CLOSED"
            status_color = (0, 0, 255)
        else:
            status = "EYES OPEN"
            status_color = (0, 255, 0)
        
        cv2.putText(frame, f"Status: {status}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, status_color, 2)
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(frame, f"Time: {int(elapsed)}s", (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        if closed_counter > 0:
            cv2.putText(frame, f"Closed frames: {closed_counter}", 
                       (10, frame.shape[0]-20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
        cv2.imshow('Enhanced Driver Fatigue Detection', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            filename = f"screenshot_{int(time.time())}.jpg"
            cv2.imwrite(filename, frame)
            print(f"Screenshot saved: {filename}")
    
    cap.release()
    cv2.destroyAllWindows()
    print("\nDetection stopped")

# Run the improved detection
run_improved_detection(duration=30)



ENHANCED DETECTION STARTED

Instructions:
  - Keep your face straight for best results
  - Ensure good lighting on your face
  - Stay 50-100cm from camera
  - Press 'q' to quit
  - Press 's' to save screenshot


Detection stopped
